In [478]:
import pandas as pd
import numpy as np
import os

In [3]:
cd '/Users/khiliv/Downloads/tables'

/Users/khiliv/Downloads/tables


In [382]:
#  READ IN TABLES 
#  NOTE: POVERTY RATE DATA COMES FROM 2015 GHANA POVERTY MAPPING REPORT PUBLISHED BY THE GHANA STATISTICAL SERVICE
#  FOUND AT http://www2.statsghana.gov.gh/docfiles/publications/POVERTY%20MAP%20FOR%20GHANA-05102015.pdf

communities = pd.read_csv('wc_communities.csv')
community_payouts = pd.read_csv('wc_community_payouts.csv')
policies = pd.read_csv('wc_customer_policies.csv')
customers = pd.read_csv('wc_customers.csv')
transactions = pd.read_csv('wc_policy_transactions.csv')
poverty = pd.read_csv('poverty.csv')

In [383]:
#  RENAME CERTAIN COLUMNS BC OF DUPLICATED NAMES ACROSS TABLES

policies = policies.rename(index=str, columns={'season': 'season_policies', 
                                              'status':'status_policies',
                                              'updated_at':'updated_at_policies',
                                              'created_at': 'created_at_policies'})

community_payouts = community_payouts.rename(index=str, columns={'season': 'season_community_payouts', 
                                              'status':'status_community_payouts',
                                              'transaction_amount':'transaction_amount_community_payouts',
                                               'created_at': 'created_at_community_payouts'})

customers = customers.rename(index=str, columns={'created_at': 'created_at_customers', 
                                              'updated_at':'updated_at_customers'})

transactions = transactions.rename(index=str, columns={'updated_at':'updated_at_transactions' })

In [384]:
#  CREATE RENEWAL VARIABLE BY LOOKING AT WHETHER A CUSTOMER HAD A POLICY IN MORE THAN ONE SEASON 

policies = policies.sort_values(by=["customer_id",'crop',"season_policies"]) 
policies['dupes'] = policies["customer_id"].duplicated(keep=False) 
policies['dupes2'] = policies[["customer_id",'season_policies']].duplicated(keep=False) 

def f(row):
    if row['dupes'] != row['dupes2']:
        val = 'renewed'
    else:
        val = 'didnt_renew'
    return val

policies['renewal'] = policies.apply(f, axis=1)

In [388]:
#  DATA CLEANING WORK FOR DISTRICT IN POVERTY DATA TO MATCH WITH CORRESPONDING VARIABLE IN COMMUNITIES DATA 

poverty['district'] = poverty['district'].str.strip()
poverty['poverty_rate'] = poverty['poverty_rate']/100

communities['district'] = communities['district'].str.replace('-',' ')
poverty['district'] = poverty['district'].str.replace('-',' ')

poverty = poverty.replace({'district':{'Nadowli-Kaleo':'Nadowli', 
                                      'Gonja Central':'Central Gonja',
                                      'Daffiama Bussie':'Daffiama Bussie Issa',
                                      'Gushiegu':'Gushegu',
                                      'Hohoe Municipal':'Hohoe',
                                      'Kasena Nankana West':'Kassena Nankana West',
                                      'Kumbumgu':'Kumbungu',
                                      'Mamprugu Moagduri':'Mamprugu Moaduri',
                                      'Sagnerigu Municipal':'Sagnarigu',
                                      'Sunyani Municipal':'Sunyani',
                                      'Tamale Metropolis':'Tamale Metropolitan',
                                      'Tatale':'Tatale Sangule',
                                      'Mamprusi West': 'West Mamprusi',
                                      'Mamprusi East': 'East Mamprusi'}})

In [390]:
#  CREATE MASTER TABLE OF CONSUMER PAYMENT LEVEL DATA, NOT DONE JUST AT CONSUMER LEVEL IN ORDER TO PRESERVE PREMIUM
#  INFORMATION

customers_m = communities.join(customers.set_index('community_id'), how='right', on='community_id')
customers_m2 = customers_m.merge(poverty,on='district', how='left')
customers_m3 = customers_m2.merge(policies, on='customer_id', how='inner')
customers_m4 = customers_m3.merge(transactions, on='customer_policy_id', how='inner')

In [391]:
#  DATA CLEANING WORK IN ORDER TO MERGE IN DISTRICT SHAPEFILE DATA FROM THE HUMANITARIAN DATA EXCHANGE FOUND AT 
#  https://data.humdata.org/dataset/ghana-administrative-boundaries 

customers_m4 = customers_m4.replace({'district':{ 'Atebubu Mamantin':'Atebubu',
                                      'Bawku West':'Bakwu West',
                                      'Bolgatanga Municipal':'Bolgatanga',
                                      'Bunkpurugu Yunyoo':'Bunkpurugu/Yunyoo',
                                      'Ejura Sekyedumase':'Ejura-Sekyedumasi',
                                      'Gushegu':'Gushiegu',
                                      'Jirapa':'Jirapa/Lambussie',
                                      'Kassena Nankana West':'Kasena/Nankani',
                                       'Saboba':'Saboba/Chereponi',
                                       'Savelugu Nanton':'Savelgu/Nanton',
                                       'Sawla Tuna Kalba':'Sawla/Tuna/Kalba',
                                       'Talensi':'Talensi-Nabdam',
                                       'Tamale Metropolitan':'Tamale',
                                        'Tolon':'Tolon/Kumbungu',
                                        'Yendi':'Yendi Municipal',
                                        'Zabzugu':'Zabzugu/Tatale'}})

In [416]:
#  CREATE SUMMARIZED COMMUNITY TABLE SHOWING SEASON WHEN PAYOUT OCCURRED IN VARIOUS COMMUNITIES AND NUMERIC DUMMY 
#  VARIABLE FOR SEASON 

community_m = community_payouts.merge(communities, on='community_id', how='left')
community2 = community_m.pivot_table(index='community_id', columns='season_community_payouts', values = 'paid_by')
com2 = pd.DataFrame(community_m.groupby(['community_id','season_community_payouts'])['transaction_amount_community_payouts'].sum())
com2 = com2.reset_index()

def f(row):
    if row['season_community_payouts'] == '2018 minor':
        val = 1
    else:
        val = 2
    return val

com2['season_payout_num'] = com2.apply(f, axis=1)

In [462]:
#  MERGE COMMUNITY PAYOUT SUMMARY FILE WITH COMMUNITY SUMMARY FILE AND CREATE NUMERIC VARIABLE FOR SEASON THAT POLICY
#  COVERED

customers_m5 = customers_m4.merge(com2, on='community_id', how='left')

def f(row):
    if row['season_policies'] == '2018 minor':
        val = 1
    elif row['season_policies'] == '2018 major':
        val = 2
    elif row['season_policies'] == '2019 minor':
        val = 3
    elif row['season_policies'] == '2019 major':
        val = 4
    else:
        val = 0
    return val

customers_m5['season_policies_num'] = customers_m5.apply(f, axis=1)

In [487]:
#  CHECK TO MAKE SURE CUSTOMERS HAD POLICY AT TIME OF POLICY PAYOUT AND ONE IN A LATER SEASON TO CREATE DUMMY VARIABLE
#  FOR RENEWAL DUE TO PAYOUT

def f(row):
    if row['season_policies_num'] == row['season_payout_num']:
        val = 1
    elif row['season_policies_num'] > row['season_payout_num']:
        val = 2
    else:
        val = 0
    return val

customers_m5['renew_payout'] = customers_m5.apply(f, axis=1)

In [488]:
#  WRITE FILE OUT TO CSV FOR USE IN TABLEAU

customers_m5.to_csv('customers_m.csv')

In [ ]:
#  TABLEAU CALCULATED VARIABLES CREATED

# Filter selection var:

# case [Filter selection]
#
# when 1 then [Crop]
# when 2 then [Payment Method]
# when 3 then [Poverty Bands]
# when 4 then [Literacy]
# when 5 then [Has Phone]
# when 6 then [Gender]
# when 7 then [Region]
#
# END


# Poverty Bands:
#
# if [Poverty Rate] < .2 then '<20%' 
# ELSEIF [Poverty Rate] >= .2 and [Poverty Rate] <.4 then '20-39%'
# ELSEIF [Poverty Rate] >= .4 and [Poverty Rate]<.6 then '40-59%'
# ELSEIF [Poverty Rate] >= .6 and [Poverty Rate] <.8 then '60-79%'
# else '>80%'
# END

# Renewed after Payout_dummy:
#
# if { fixed [Customer Id]: sum([Renew Payout])} > 2 then 1 else 0 end

# Renewal (customer level):
# 
# {fixed [Customer Id]: max([Renewal (num)])}

# Renewal (numeric):
#
# If [Renewal] = 'renewed' then 1 else 0 END

# Renewal After Payout:
#
# { fixed [Customer Id]: sum([Renew Payout])}

# Renewal Rate (District):
#
# {FIXED [District]: sum([Renewal (customer level)])/countd([Customer Id])}

# 